In [1]:
# Step 1: Load an experiment configuration from a JSON file
import json
import os
import glob
from scripts.sample_db import SampleDB
from scripts.config_model import Experiment, load_experiment_config, save_experiment_config, update_experiment_config


In [2]:
# Inputs

# sample root path 
root_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-27\f3'

# database path
db_file_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'



In [3]:
# Loading experiment configuration
config_file_path = glob.glob(os.path.join(root_path, 'config_*.json'))[0]
print(config_file_path)

config = load_experiment_config(config_file_path)
print("Loaded experiment configuration:")
print(config.paths)

# Define your directories here
trials_path = os.path.join(root_path, "trials")
anatomy_path = os.path.join(root_path, "anatomy")
raw_trials_path = os.path.join(trials_path,"raw")

# Add to config structure
config.paths.root_path = root_path
config.paths.config_path = config_file_path
config.paths.anatomy_path = anatomy_path
config.paths.trials_path = trials_path

save_experiment_config(config)

# Step 2: Load or create a sample database
sample_db = SampleDB()

try:
    sample_db.load(db_file_path)
    print("Loaded existing sample database.")
except FileNotFoundError:
    print("Sample database not found. Creating a new one.")
    sample_db.save(db_file_path)  # Create a new CSV file with headers

sample = config.sample
print(sample.id)
existing_sample = sample_db.get_sample(sample.id)


# Step 3: Check if the sample is already in the database and ask if you want to update it
update=True
if existing_sample:
    print(f"Sample with ID {sample.id} already exists.")
    update = input("Do you want to update the existing sample? (y/n): ").strip().lower() == 'y'
    print(update)
# Ensure initial columns are present
sample_db.add_column('sample_id')
sample_db.add_column('root_path')
sample_db.add_column('config_path')
sample_db.add_column('trials_path')
sample_db.add_column('anatomy_path')
sample_db.add_column('em_path')
sample_db.add_column('update')


# Add the sample data to the database
sample_db.samples[sample.id] = {
    "sample": sample,
    "root_path": root_path,
    "config_path": config_file_path,
    "trials_path": trials_path,
    "anatomy_path": anatomy_path,
    "em_path": "",
    "update": update
}

# Add columns for each processing step
processing_steps = [
    '00_load_experiment',
    '01_register_lm_trials',
    '02_register_lm_trials_lm_stack',
    '03_segment_lm_trials',
    '10_extract_lm_traces',
    '11_normalize_lm_traces',
    '12_deconvolve_lm_traces',
    '13_analyze_correlation_lm_traces',
    '20_preprocess_lm_stack',
    '21_register_lm_stack_to_ref_stack',
    '22_segment_lm_stack_from_em_warped_stack',
    '23_extract_marker_from_channel',
    '30_segment_em_stack',
    '31_segment_glomeruli',
    '32_find_landmarks_with_BigWarp',
    '33_register_em_stack_lm_stack_from_landmarks',
    '34_register_em_stack_lm_trials'
]
for step in processing_steps:
    sample_db.samples[sample.id][step] = False
    
# Step 4: Save the updated database to a file
sample_db.save(db_file_path)
print(f"Sample database saved to {db_file_path}.")

# Update the '00_load_experiment' step to True
sample_db.samples[sample.id]['00_load_experiment'] = True
sample_db.save(db_file_path)

# Function to print the current status of a sample
def print_sample_status(sample_id):
    sample = sample_db.samples.get(sample_id)
    if sample:
        print(f"Status for sample {sample_id}:")
        for step in processing_steps:
            status = "Completed" if sample[step] else "Pending"
            print(f"  {step}: {status}")
    else:
        print(f"Sample {sample_id} not found in database.")

# Print the current status of the sample
print_sample_status(sample.id)

\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-27\f3\config_20220427_RM0008_126hpf_fP3_f3.json
Loaded experiment configuration:
root_path=WindowsPath('//tungsten-nas.fmi.ch/tungsten/scratch/gfriedri/montruth/2P_RawData/2022-04-27/f3') config_path=WindowsPath('//tungsten-nas.fmi.ch/tungsten/scratch/gfriedri/montruth/2P_RawData/2022-04-27/f3/config_20220427_RM0008_126hpf_fP3_f3.json') trials_path=WindowsPath('//tungsten-nas.fmi.ch/tungsten/scratch/gfriedri/montruth/2P_RawData/2022-04-27/f3/trials') anatomy_path=WindowsPath('//tungsten-nas.fmi.ch/tungsten/scratch/gfriedri/montruth/2P_RawData/2022-04-27/f3/anatomy') em_path=WindowsPath('.')
\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-27\f3\config_20220427_RM0008_126hpf_fP3_f3.json
Loaded existing sample database.
20220427_RM0008_126hpf_fP3_f3
Sample with ID 20220427_RM0008_126hpf_fP3_f3 already exists.
False
Sample database saved to \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri

In [4]:
sample_db

SampleDB(sample_ids=['20220426_RM0008_130hpf_fP1_f3', '20220118_RM0012_124hpf_fP8_f2', '20220427_RM0008_126hpf_fP3_f3'])